## CHIRPS COGs
### better-wfp-02-04-01

This application takes a CHIPS products, performs COGs and return the optimized product togheter with a .yml file

### <a name="service">Service definition

In [ ]:
service = dict([('title', 'CHIRPS COGs'),
                ('abstract', 'CHIRPS COGs'),
                ('id', 'ewf-wfp-02-04-01')])

### <a name="runtime">Runtime parameter definition

**Input identifiers**

In [ ]:
input_identifiers = ['chirps-v2.0.2019.09.10']

**Input references**

In [ ]:
input_references = ['https://catalog.terradue.com/chirps/search?uid=chirps-v2.0.2019.09.10']

**Data path**

This path defines where the data is staged-in. 

In [ ]:
data_path = '/workspace/data'

### <a name="workflow">Workflow

#### Import the packages required for processing the data

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import yaml

sys.path.append('/application/notebook/libexec/') 
sys.path.append(os.getcwd())

from ellip_helpers import *
from uuid import uuid4
from shapely.wkt import loads

import gdal

import cioppy
ciop = cioppy.Cioppy()

## Read the products

### check if all the products have the same track number

In [ ]:
metadata = dict()

try:
    metadata = ciop.search(end_point=input_references[0],
                         params=dict(),
                         output_fields='',
                         model='GeoTime')[0]
    
    print metadata

except Exception as e:
    print('ERROR: could not retrieve products metadata. {}'.format(e))

In [ ]:
metadata

### Copy the input in the local folder

In [ ]:
product_name = input_identifiers[0]+'.tif'

product_name

In [ ]:
os.listdir(os.getcwd())

In [ ]:
product_path = os.path.join(data_path, product_name)

product_path

In [ ]:
path = ciop.copy(product_path, os.getcwd())

path

In [ ]:
os.listdir(os.getcwd())

In [ ]:
os.rename(product_name, '{}_{}'.format('temp', product_name))

In [ ]:
os.listdir(os.getcwd())

In [ ]:
print gdal.Info('temp_' + product_name)

### Apply Cog

In [ ]:
cog('temp_' + product_name, product_name, clear=True)

In [ ]:
os.listdir(os.getcwd())

In [ ]:
print gdal.Info(input_identifiers[0] + '.tif')

### Extract projection and geo_transform

In [ ]:
ds = gdal.Open(product_name)

In [ ]:
metadata['geo_transform'] = ds.GetGeoTransform()
metadata['projection'] = ds.GetProjection()
ds.FlushCache()

In [ ]:
metadata

### Create yaml file

In [ ]:
metadata['valid_data'] = [[-127.56829548716837, 50.0], [-178.5499711456919, 28.451697453558694], [-180.0, -14.227485158242985], [-180.0, -19.477637468425044],
                          [-177.14966157616104, -43.85580860396533], [-176.92428210072717, -44.1937092674017], [-176.47428209402165, -44.44370927112699],
                          [-76.54920491878684, -50.00000149011612], [-32.97103354229944, -50.00000149011612], [179.04997943956363, -49.75161095788991],
                          [180.00000536441803, -20.253305991160417], [180.00000536441803, -10.005885337836311], [179.99612502572722, -9.33068827589868],
                          [155.19609441497164, 49.669384662964575], [154.7723656682719, 49.89472135798122], [147.63822384433456, 50.0], [-127.56829548716837, 50.0]]

In [ ]:
yaml_file = {
            'creation_dt':metadata['startdate'][0:10],
            'extent':{
                'center_dt':metadata['startdate'][0:10],
                'coord':{
                    'ul':{
                        'lon': list(loads(metadata['wkt']).exterior.coords)[3][0],
                        'lat': list(loads(metadata['wkt']).exterior.coords)[3][1],
                    },
                    'ur':{
                        'lon': list(loads(metadata['wkt']).exterior.coords)[2][0],
                        'lat': list(loads(metadata['wkt']).exterior.coords)[2][1],
                    },
                    'll':{
                        'lon': list(loads(metadata['wkt']).exterior.coords)[0][0],
                        'lat': list(loads(metadata['wkt']).exterior.coords)[0][1],
                    },
                    'lr':{
                        'lon': list(loads(metadata['wkt']).exterior.coords)[1][0],
                        'lat': list(loads(metadata['wkt']).exterior.coords)[1][1],
                    },
                },
                'from_dt':metadata['startdate'][0:10],
                'to_dt':metadata['enddate'][0:10],
                },
            'format':{'name':'GeoTiff'},
            'grid_spatial':{
                'projection':{
                    'geo_ref_points':{
                        'ul':{
                            'x': list(loads(metadata['wkt']).exterior.coords)[3][0],
                            'y': list(loads(metadata['wkt']).exterior.coords)[3][1],
                        },
                        'ur':{
                            'x': list(loads(metadata['wkt']).exterior.coords)[2][0],
                            'y': list(loads(metadata['wkt']).exterior.coords)[2][1],
                        },
                        'll':{
                            'x': list(loads(metadata['wkt']).exterior.coords)[0][0],
                            'y': list(loads(metadata['wkt']).exterior.coords)[0][1],
                        },
                        'lr':{
                            'x': list(loads(metadata['wkt']).exterior.coords)[1][0],
                            'y': list(loads(metadata['wkt']).exterior.coords)[1][1],
                        },
                    },
                    'spatial_reference':metadata['projection'],
                    'valid_data':{
                        'coordinates':metadata['valid_data'],
                        'type':'Polygon',
                        },
                    },
                },
            'id': str(uuid4()),
            'image':{
                'bands':{'rfe':{'path':product_name}}
            },
            'instrument':{'name':'VIRS'},
            'lineage': {'source_datasets':'CHIRPS'},
            'platform':{'code':'TRMM'},
            'processing_level':'ARD',
            'product_type':'chirps_daily',
        }

In [ ]:
yaml_file

In [ ]:
with open('metadata.yaml', 'w') as outfile:
    yaml.dump(yaml_file, outfile, default_flow_style=False)

### Metadata

In [ ]:
# metadata for product

create_metadata(metadata, metadata['identifier'])

# metadate for yaml file

metadata['identifier'] = 'metadata.yaml'
metadata['title'] = 'metadata yaml file'

create_metadata(metadata, metadata['identifier'])

### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.